In [ ]:
from fastai.vision.all import *

from pathlib import Path

import torch
import torch.nn as nn
import torchvision.transforms as T
import torch.functional as F

from modules_th.video_block import *
from modules_th.inflator import *

/home/adrian/anaconda3/envs/VE/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729047590/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
path = Path(r'/mnt/data/eugeniomarinelli/UCF_Crimes/Videos')

In [ ]:
vds = get_files(path, extensions='.mp4')
vid_path = vds[0]
vid_tens = create_video_tensor(vid_path)
vid_tens.shape

(3, 30, 240, 320)

In [ ]:
vds = get_files(path, extensions='.mp4')
lbls = vds.map(parent_label)
vid_tfms = [partial(create_video_tensor, l=30)]
lbl_tfms = [parent_label, Categorize()]

n_lbl, n_el = 4, 4
splits = RandomSplitter(valid_pct=0.2, seed=42)(vds)
splits = [uniformize_dataset(idxs, lbls[idxs], n_lbl=n_lbl, n_el=n_el) for idxs in splits]
ds = Datasets(vds, [vid_tfms, lbl_tfms], splits=splits)

dls = ds.dataloaders(bs=n_lbl*n_el, 
                     after_item=[Resize(128)],
                     after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])

In [ ]:
import time
t = time.perf_counter()
x, y = dls.one_batch()
time.perf_counter() - t

45.994558528997004

In [ ]:
body = sequential(ResBlock(1, 3, 64, ks=7, stride=3, act_cls=Mish, bias=False),
                  ResBlock(1, 64, 128, stride=2, act_cls=Mish),
                  ResBlock(1, 128, 256,  stride=2, act_cls=Mish),
                  ResBlock(1, 256, 128, stride=2, act_cls=Mish),
                  ResBlock(1, 128, 64,  stride=2, act_cls=Mish)
                 )

head = sequential(AdaptiveConcatPool2d(1),
                  Flatten(),
                  LinBnDrop(128, 128, lin_first=True),
                  mish,
                  nn.Linear(128, 10)
                 )

m = inflate(sequential(body,head)).eval()

In [ ]:
x.cpu()
n_embs = tensor(body(x).shape)[1:].prod().item()
body(x).shape, m(x).shape

(torch.Size([32, 64, 1, 5, 5]), torch.Size([32, 10]))

In [ ]:
learn = Learner(dls, m, metrics=accuracy)

In [ ]:
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.939604,1.114729,0.649427,00:15
1,0.914461,1.041761,0.677452,00:21
2,0.807988,0.889782,0.724841,00:22
3,0.707722,0.728764,0.787771,00:20
4,0.621713,0.710635,0.791338,00:15


In [ ]:
head = torch.nn.MultiheadAttention(1024, 8)
z = torch.randn(128,1,1024)
x, y = head(z,z,z)
x.shape, y.shape

(torch.Size([128, 1, 1024]), torch.Size([1, 128, 128]))

In [ ]:
vds = get_files(Path(r'/mnt/data/eugeniomarinelli/UCF_Crimes/Videos'), extensions='.mp4')
vds[0]

Path('/mnt/data/eugeniomarinelli/UCF_Crimes/Videos/Fighting/Fighting007_x264.mp4')

In [ ]:
vid = vds[0]
parent_label(vid)
vid_tens = create_video_tensor(vid, l=200)

In [ ]:
vid_tens.shape

torch.Size([3, 1264, 240, 320])

In [ ]:
?? Resize